In [1]:
import os
import pdf2image
from PIL import Image
import pytesseract
import difflib
import re
import pandas as pd
import sys
sys.path.append('../')
from helper import *

In [2]:
DATA_PATH = '../../data/'
STATE = 'tripura/'

PARSE_DATA_PAGES = "../../parseData/images/"+STATE
create_path(PARSE_DATA_PAGES)

PARSE_DATA_BLOCKS = "../../parseData/blocks/"+STATE
create_path(PARSE_DATA_BLOCKS)

PARSE_DATA_CSVS = "../../parseData/csvs/"+STATE
create_path(PARSE_DATA_CSVS)

COLUMNS = ["number","id", "elector_name", "father_or_husband_name", "relationship", "house_no", "age", "sex", "ac_name", "parl_constituency", "part_no", "year", "state", "filename", "main_town", "police_station", "mandal", "revenue_division", "district", "pin_code", "polling_station_name", "polling_station_address", "net_electors_male", "net_electors_female", "net_electors_third_gender", "net_electors_total"]

state_pdfs_path = DATA_PATH+STATE
state_pdfs_files = os.listdir(state_pdfs_path)

In [3]:
state_pdfs_files

['ac0570038.pdf',
 'ac0520034.pdf',
 'ac0520027.pdf',
 'ac0470029.pdf',
 'ac0590043.pdf',
 'ac0250014.pdf',
 'ac0140025.pdf',
 'ac0450028.pdf',
 'ac0410008.pdf',
 'ac0380042.pdf',
 'ac0480026.pdf',
 'ac0070042.pdf',
 'ac0410032.pdf',
 'ac0270037.pdf',
 'ac0550034.pdf',
 'ac0510001.pdf',
 'ac0550024.pdf',
 'ac0410012.pdf',
 'ac0440036.pdf',
 'ac0570026.pdf',
 'ac0440057.pdf',
 'ac0600033.pdf',
 'ac0160007.pdf',
 'ac0570021.pdf',
 'ac0460030.pdf']

In [4]:
def split_data(data):
    seps = [":",".",","," "]
    
    for s in seps:
        if s in data:
            break

    data = data.split(s)
    data = [ i for i in data if i.strip()!='']
    if len(data)>1:
        data = data[1].strip()
        return data
    else:
        data = ""

In [5]:
def extract_4_numbers(crop_stat_path):
    
    text = (pytesseract.image_to_string(crop_stat_path, config='--psm 6', lang='eng')) #config='--psm 4' config='-c preserve_interword_spaces=1'

    text = re.findall(r'\d+', text)

    if len(text)==4:
        if int(text[0]) + int(text[1]) == int(text[2]):
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[2]
        elif int(text[0]) + int(text[1]) == int(text[3]):
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[3]
        else:
            net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],text[2],text[3]
    elif len(text) == 3 and int(text[2])>=int(text[1]) and int(text[2])>=int(text[0]):
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],text[1],"0",text[2]
    elif len(text) == 2 and int(text[0])*2-100<int(text[1]):
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = text[0],int(text[1])-int(text[0]),"0",text[1]
    else:
        net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = "","","",""
    
    return net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total

def extract_first_page_details(path):
    
    img = Image.open(path)
        
    a,b,c,d = 1512,5094,2076,131  # stats for male and female
    crop_img = crop_section(a,b,c,d,img)

    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    crop_stat_path = crop_path+"stat.jpg"
    crop_img.save(crop_stat_path)
    
    a_n,b_n,c_n,d_n = extract_4_numbers(crop_stat_path)
        
    
    a,b,c,d = 2515,1875,1227,1631  # mandal block
    crop_img = crop_section(a,b,c,d,img)
    
    crop_det_path = crop_path+"det.jpg"
    crop_img.save(crop_det_path)

    text = (pytesseract.image_to_string(crop_det_path, config='--psm 6', lang='eng+ben')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    main_town,police_station,revenue_division,mandal,district,pin_code = "","","","","",""
    
    if len(text) == 6:
        
        main_town = split_data(text[0])
        police_station = split_data(text[2])
        revenue_division = split_data(text[3])
        mandal = split_data(text[1])
        district = split_data(text[4])
        pin_code = split_data(text[5])
    
    elif len(text) == 5:
        
        main_town = split_data(text[0])
        police_station = split_data(text[2])
        revenue_division = split_data(text[3])
        mandal = split_data(text[1])
        district = ''
        pin_code = split_data(text[4])
        
    elif len(text) == 7:
        
        main_town = split_data(text[0])
        police_station = split_data(text[3])
        revenue_division = split_data(text[4])
        mandal = split_data(text[2])
        district = split_data(text[5])
        pin_code = split_data(text[6])
        
 
    a,b,c,d = 3021,330,604,212 # part no
    crop_img = crop_section(a,b,c,d,img)
    
    crop_part_path = crop_path+"part.jpg"
    crop_img.save(crop_part_path)

    text = (pytesseract.image_to_string(crop_part_path, config='--psm 6', lang='eng+ben')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = re.findall(r'\d+', text)
    
    if len(text)>0:
        part_no = text[0]
    else:
        part_no = ""
            
        
    a,b,c,d = 328,3874,1850,714 # police name name and address
    crop_img = crop_section(a,b,c,d,img)
    
    crop_police_path = crop_path+"police.jpg"
    crop_img.save(crop_police_path)
    
    text = (pytesseract.image_to_string(crop_police_path, config='--psm 6', lang='eng+ben')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    if len(text) == 4:
        polling_station_name = text[1].strip()
        polling_station_address = text[3].strip()
    elif len(text) == 5:
        polling_station_name = text[1].strip()
        polling_station_address = text[3]+" "+text[4]
    else:
        polling_station_name, polling_station_address = "",""
    
        
    a,b,c,d = 275,300,2250,564 # ac name and parl
    crop_img = crop_section(a,b,c,d,img)
    
    crop_ac_path = crop_path+"ac.jpg"
    crop_img.save(crop_ac_path)
    
    text = (pytesseract.image_to_string(crop_ac_path, config='--psm 6', lang='eng+ben')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    ac_name,parl_constituency = '',''

    if len(text) == 4:
        
        try:
            seps = [":"]

            for s in seps:
                if s in text[0]:
                    break
            text[1] = text[1].replace("-:",'')
            ac_name = text[1].split(s)[1]
        except:
            ac_name = ''
        
        
        try:
            seps = [":"]

            for s in seps:
                if s in text[1]:
                    break
            parl_constituency = text[3].split(s)[1]
        except:
            parl_constituency = ''
        
    
    return [ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,a_n,b_n,c_n,d_n]



In [6]:
def generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page):
    
    img = Image.open(page_full_path)

    amend = False
    
    def generate(intial_width,a,b,gap):
        count = 0
        crop_width = 1220
        crop_height = 480

        for col in range(1,11):

            for row in range(1,4):
                c = a+crop_width
                d = b+crop_height
                area = (a, b, c, d)
                cropped_img = img.crop(area)
                count = count+1
                cropped_img.save(page_blocks_path+str(count)+".jpg")

                a = c

            a = intial_width
            b = b+crop_height+gap
    
    page_type,intial_height = check_page_type(img,amend_page)
    
    if page_type == 1:
        intial_width = 120
        generate(intial_width,intial_width,intial_height,5)

        

def check_page_type(img,amend_page):
    
    return 1,495

    
        
def step2_order(params):
    
    if len(params) == 6:
        new_params_list = []
        
        new_params_list.append(params[0] + " " + params[1])
       
        for i in params[2:]:
            new_params_list.append(i)
            
        return new_params_list

    return params



In [69]:
def getAG(new_params_list):
    if len(new_params_list)>=5:
        return new_params_list[-2],new_params_list[-1]
    else:
        return "",""
def getN(new_params_list):
    if len(new_params_list)>0 and len(new_params_list[0])<=2:
        return new_params_list[0]
    else:
        return ""
    
def getRel(new_params_list):
    
    if len(new_params_list)>5 :
        return new_params_list[3]
    else:
        return ""
    

In [76]:
def extract_name(name):
    
    row = name.split(":")
    if len(row)!=2:
        return ""
    else:
        return row[1].strip()
    
def extract_id(id):
    row = id.split(" ")
    if len(row)>0:
        return row[-1].strip()

    else:
        return "",""

def extract_house_no(house_no):
    row = house_no.split(":")
    if len(row)==2:
         return row[1].strip()
    else:
        return ""
    
def extract_gender(gender):
    row = gender.split(":")
    
    if len(row)==2:
        
        if 'পুং' in row[1]:
            return "Male"
        if 'স্ত্রী' in row[1]:
            return "Female"
        else:
            return "Male"
        
    else:
        return ""


def extract_rel_name(rel_name):
    row = rel_name.split(":")
    if len(row)!=2:
        
        row = rel_name.split(";")
        if len(row)!=2:
            return "",""
        else:
            rel_type = extract_rel_type(row[0].strip())
            return row[1].strip(),rel_type
    else:
        rel_type = extract_rel_type(row[0].strip())
        
        return row[1].strip(),rel_type
    
def extract_rel_type(rel_type):
    line = rel_type
    if line.startswith("স্বামী"):
        rel_type = 'husband'
    elif line.startswith("পিতা"):
        rel_type = 'father'
    else:
        rel_type = ""
    
    return rel_type 


def extract_details_from_block(block):
        
    id = block[0]
    name = block[1]
    rel_name = block[2]
    house_no = block[3]
    age = block[4]
    gender = block[5]
    number = block[6]
    
    name = extract_name(name)
    rel_name,rel_type = extract_rel_name(rel_name)
    house_no = extract_house_no(house_no)
    gender = extract_gender(gender)
    voter_id = extract_id(id)
    
    
    return [name,rel_name,rel_type,house_no,age,gender,voter_id,number]


In [77]:
def arrange_columns(first_page_list,block_list,filename):
    
    year = 2018
    state = 'tripura'
    
    ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = first_page_list
    name,rel_name,rel_type,house_no,age,gender,voter_id,number = block_list
    
    final_list = [number,voter_id,name,rel_name,rel_type,house_no,age,gender,ac_name,
                 parl_constituency,part_no,year,state,filename,main_town,police_station,mandal,
                 revenue_division,district,pin_code,polling_station_name,polling_station_address,
                 net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total]
    
    
    return final_list


In [80]:
for pdf_file_name in ['ac0470029.pdf']:
# for pdf_file_name in state_pdfs_files:
    
    print(pdf_file_name)
    
    #create images,blocks and csvs paths for each file
    pdf_file_name_without_ext = pdf_file_name.split('.pdf')[0]
    input_pdf_images_path = PARSE_DATA_PAGES+pdf_file_name_without_ext+"/"
    create_path(input_pdf_images_path)

    input_images_blocks_path = PARSE_DATA_BLOCKS+pdf_file_name_without_ext+"/"
    create_path(input_images_blocks_path)
    
    #convert pdf into bunch of images
    pdf_2_images_list = pdf_to_img(state_pdfs_path+pdf_file_name, input_pdf_images_path,dpi=500)
    
  
    
    #sort pages for looping
    input_images = os.listdir(input_pdf_images_path)
    sort_nicely(input_images)
    
    #empty intial data
    df = pd.DataFrame(columns = COLUMNS)
    order_problem = []
    
    amend_page = False
    
    #for each page, parse the data
    for page in input_images:
    
        page_full_path = input_pdf_images_path+page
        
        #extract first page content
        if page == '1.jpg':
            first_page_list = extract_first_page_details(page_full_path)
            continue
            

        #ingnore 2nd page and last page
        if page == '2.jpg' or input_images[-1] == page:
            continue
        
#         if os.path.exists(PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv"):
#             print(pdf_file_name_without_ext+".csv", "already exists")
#             break
            

        #loop from 3 page onwards
        if page.endswith('.jpg'):
            
            final_invidual_blocks = []
            blocks_path = input_images_blocks_path+"blocks/"
            create_path(blocks_path)

            page_idx = page.split(".jpg")[0] + "/"
            page_blocks_path = blocks_path+page_idx
            create_path(page_blocks_path)
                
            generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page)
            
            sorted_blocks = os.listdir(page_blocks_path)
            sort_nicely(sorted_blocks)
            
            for jpg_file in sorted_blocks:
                
                if jpg_file.endswith('.jpg') :

                    text = (pytesseract.image_to_string(page_blocks_path+jpg_file, config='--psm 6', lang='eng+ben')) #config='--psm 4' config='-c preserve_interword_spaces=1'
                    params_list = text.split('\n')
                    params_list = [ i for i in params_list if i!='' and i!='\x0c']
                    temp_list = params_list.copy()
                    
                    text = (pytesseract.image_to_string(page_blocks_path+jpg_file, config='--psm 11', lang='eng+ben')) #config='--psm 4' config='-c preserve_interword_spaces=1'
                    new_params_list = text.split('\n')
                    new_params_list = [ i for i in new_params_list if i!='' and i!='\x0c']
                    
                    if len(temp_list)==4:
                        a,g = getAG(new_params_list)
                        
                        params_list.append(a)
                        params_list.append(g)
                        
                        n = getN(new_params_list)
                        params_list.append(n)

                        
                    elif len(temp_list) == 3:
                        rel = getRel(new_params_list)
                        a,g = getAG(new_params_list)
                        temp = params_list[2]
                        params_list[2] = rel
                        params_list.append(temp)
                        params_list.append(a)
                        params_list.append(g)
                        
                        n = getN(new_params_list)
                        params_list.append(n)
                        
                    if len(params_list) == 7:
                        final_invidual_blocks.append(params_list)
                    else:
                        order_problem.append((page, jpg_file,params_list))
        
        #put the data into dataframe
        for block in final_invidual_blocks:
            
            block_list = extract_details_from_block(block)
                        
            final_list = arrange_columns(first_page_list,block_list,pdf_file_name_without_ext)            
            df_length = len(df)
            df.loc[df_length] = final_list
        
        print("page done : ",page)
                
    #save the dataframe(pdf) data into csv
    save_to_csv(df,PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv")
    print("CSV saved")
    print()


ac0470029.pdf
   number                id            elector_name father_or_husband_name  \
0          TP/02/048/027061        মঙ্গল চন্দ্র কলই             নরসিংহ কলই   
1          TP/02/048/027046         সন্ধ্যারানী কলই       মঙ্গল চন্দ্র কলই   
2                GKD0191692              জহরলাল কলই       মঙ্গল চন্দ্র কলই   
3                YES0111310                সমীর কলই           শুক্রুপদ কলই   
4                GKD0235978                উমেশ কলই       মঙ্গল চন্দ্র কলই   
5                GKD0236000                রীনা কলই                          
6                GKD0340091        লাল এং জামি মলসম               উমেশ কলই   
7                YES0111328             বাসন্তী কলই                          
8                GKD0343608             সুশান্ত কলই       মঙ্গল চন্দ্র কলই   
9      10  TP/02/048/027003             তিলকতমা কলই           বিশ্বমনি কলই   
10     11        GKD0266205             গৌবিন্দ কলই           বিশ্বমনি কলই   
11     12        GKD0263863                বিমল কল

   number                id            elector_name father_or_husband_name  \
0          TP/02/048/027061        মঙ্গল চন্দ্র কলই             নরসিংহ কলই   
1          TP/02/048/027046         সন্ধ্যারানী কলই       মঙ্গল চন্দ্র কলই   
2                GKD0191692              জহরলাল কলই       মঙ্গল চন্দ্র কলই   
3                YES0111310                সমীর কলই           শুক্রুপদ কলই   
4                GKD0235978                উমেশ কলই       মঙ্গল চন্দ্র কলই   
5                GKD0236000                রীনা কলই                          
6                GKD0340091        লাল এং জামি মলসম               উমেশ কলই   
7                YES0111328             বাসন্তী কলই                          
8                GKD0343608             সুশান্ত কলই       মঙ্গল চন্দ্র কলই   
9      10  TP/02/048/027003             তিলকতমা কলই           বিশ্বমনি কলই   
10     11        GKD0266205             গৌবিন্দ কলই           বিশ্বমনি কলই   
11     12        GKD0263863                বিমল কলই           বি

   number                id         elector_name father_or_husband_name  \
0          TP/02/048/027061     মঙ্গল চন্দ্র কলই             নরসিংহ কলই   
1          TP/02/048/027046      সন্ধ্যারানী কলই       মঙ্গল চন্দ্র কলই   
2                GKD0191692           জহরলাল কলই       মঙ্গল চন্দ্র কলই   
3                YES0111310             সমীর কলই           শুক্রুপদ কলই   
4                GKD0235978             উমেশ কলই       মঙ্গল চন্দ্র কলই   
..    ...               ...                  ...                    ...   
77     80        GKD0343137  অংইয়াক রায় রিয়াং       মঙ্গল হাম রিয়াং   
78     81        GKD0343087        খবিরুং রিয়াং                          
79     82        YES0033720          অমৃত রিয়াং                          
80     83        YES0033738      কুকতিরুং রিয়াং                          
81     84        YES0023358        কমলজয় রিয়াং         Saar aa রিয়াং   

   relationship house_no age     sex                        ac_name  \
0        father        1  62

    number                id            elector_name father_or_husband_name  \
0           TP/02/048/027061        মঙ্গল চন্দ্র কলই             নরসিংহ কলই   
1           TP/02/048/027046         সন্ধ্যারানী কলই       মঙ্গল চন্দ্র কলই   
2                 GKD0191692              জহরলাল কলই       মঙ্গল চন্দ্র কলই   
3                 YES0111310                সমীর কলই           শুক্রুপদ কলই   
4                 GKD0235978                উমেশ কলই       মঙ্গল চন্দ্র কলই   
..     ...               ...                     ...                    ...   
137         TP/02/048/000076    সুকু চন্দ্র দেববর্মা                          
138         TP/02/048/000069      নন্দ রানী দেববর্মা    সুকুচন্দ্র দেববর্মা   
139               YES0079277            ধরম দেববর্মা                          
140               YES0079285  রেবিকা মলসম (দেববর্মা)                          
141               YES0079293         সঞ্জয় দেববর্মা                          

    relationship house_no age     sex              

    number                id                 elector_name  \
0           TP/02/048/027061             মঙ্গল চন্দ্র কলই   
1           TP/02/048/027046              সন্ধ্যারানী কলই   
2                 GKD0191692                   জহরলাল কলই   
3                 YES0111310                     সমীর কলই   
4                 GKD0235978                     উমেশ কলই   
..     ...               ...                          ...   
174               YES0129270  চন্দ্র সোনা মলসম (জমাতিয়া)   
175               YES0144386       নরেন্দ্রকন্যা জমাতিয়া   
176               GKD0365411                কানাইলাল মলসম   
177               GKD0365429                বর্ষাহাম মলসম   
178               YES0129239              উপেন লক্ষী মলসম   

    father_or_husband_name relationship house_no age     sex  \
0               নরসিংহ কলই       father        1  62    Male   
1         মঙ্গল চন্দ্র কলই      husband        1  58  Female   
2         মঙ্গল চন্দ্র কলই       father        1  40    Male   
3          

    number                id      elector_name father_or_husband_name  \
0           TP/02/048/027061  মঙ্গল চন্দ্র কলই             নরসিংহ কলই   
1           TP/02/048/027046   সন্ধ্যারানী কলই       মঙ্গল চন্দ্র কলই   
2                 GKD0191692        জহরলাল কলই       মঙ্গল চন্দ্র কলই   
3                 YES0111310          সমীর কলই           শুক্রুপদ কলই   
4                 GKD0235978          উমেশ কলই       মঙ্গল চন্দ্র কলই   
..     ...               ...               ...                    ...   
228               GKD0361675   নারা মনি রিয়াং                          
229               GKD0361279   রাংখুংতি রিয়াং                          
230               YES0117416      জনাধন রিয়াং                          
231               YES0117424     পানবতী রিয়াং          সিজরাম রিয়াং   
232               YES0003616    রাসাইহা রিয়াং                নারামনি   

    relationship house_no age     sex                        ac_name  \
0         father        1  62    Male   - 47 - আমবা

    number                id      elector_name father_or_husband_name  \
0           TP/02/048/027061  মঙ্গল চন্দ্র কলই             নরসিংহ কলই   
1           TP/02/048/027046   সন্ধ্যারানী কলই       মঙ্গল চন্দ্র কলই   
2                 GKD0191692        জহরলাল কলই       মঙ্গল চন্দ্র কলই   
3                 YES0111310          সমীর কলই           শুক্রুপদ কলই   
4                 GKD0235978          উমেশ কলই       মঙ্গল চন্দ্র কলই   
..     ...               ...               ...                    ...   
258               GKD0264077      রবিরং রিয়াং                          
259         TP/02/048/027077       মতিলাল মলসম                          
260               GKD0264242      প্রমিলা মলসম            মতিলাল মলসম   
261               GKD0191502  বৃন্দাজয় রিয়াং                          
262               GKD0263996      গনরুং রিয়াং                          

    relationship house_no age     sex                        ac_name  \
0         father        1  62    Male   - 47 - আমবা

In [81]:
#combine all state files into one csv
combine_all_csvs("tripura_final.csv",PARSE_DATA_CSVS)